In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models, transforms,datasets

from PIL import Image
import os
import numpy as np
import pickle

In [2]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()

In [3]:
# Load train-test list
trainList = []
testList = []
train_path = "C:\\Users\\smitr\\Desktop\\train_10class\\"
test_path  = "C:\\Users\\smitr\\Desktop\\test_10class\\"
classes = os.listdir("C:\\Users\\smitr\\Desktop\\train_10class\\")

for class_ in classes:
    trainList.extend(os.listdir(train_path+class_))
    testList.extend(os.listdir(test_path+class_))
    
print(len(trainList),len(testList))

1097 277


In [5]:
net = models.resnet18()
net.fc = nn.Linear(512,10)
net.load_state_dict(torch.load('resnetPre_fcOnly10class_ucf101_10adam_1e-4_b10.pt'))
# Removing fully connected layer for feature extraction
net = nn.Sequential(*list(net.children())[:-1])

In [5]:
print(list(net.children()))

[Sequential(
  (0): ConvBNReLU(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
      )
      (1): C

)]


In [6]:
# Count no.of trainable parameters
total_params = 0
for params in net.parameters():
    print(params.size())
    total_params += np.sum(np.prod(params.size()))
print("No.of parameters: ",total_params)

torch.Size([64, 3, 7, 7])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([128, 64, 3, 3])
torch.Size([128])
torch.Size([128])
torch.Size([128, 128, 3, 3])
torch.Size([128])
torch.Size([128])
torch.Size([128, 64, 1, 1])
torch.Size([128])
torch.Size([128])
torch.Size([128, 128, 3, 3])
torch.Size([128])
torch.Size([128])
torch.Size([128, 128, 3, 3])
torch.Size([128])
torch.Size([128])
torch.Size([256, 128, 3, 3])
torch.Size([256])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([256])
torch.Size([256, 128, 1, 1])
torch.Size([256])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([256])
torch.Size([512, 256, 3, 3])
torch.Size([512])
t

In [7]:
if use_gpu:
    print("GPU is available !!!")
    net = net.cuda()

GPU is available !!!


<h2> Feature Extraction </h2>

In [8]:
# PIL image to tensor transformation
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),            
        transforms.ToTensor()
    ])

In [11]:
# Creating features directory
dir_name = "C:\\Users\\smitr\\Desktop\\ucf10_resnetFeat"
os.mkdir(dir_name)
os.mkdir(dir_name+"\\train")
os.mkdir(dir_name+"\\test")

In [10]:
import time

In [12]:
framePath = "C:\\Users\\smitr\\Desktop\\train_10class\\"
dir_name = "C:\\Users\\smitr\\Desktop\\ucf10_resnetFeat"
item_done = []
start = time.time()

for item in trainList:
    cName = item.split('_')[1]
    srcPath = framePath+cName+'\\'+item    
    fNames = os.listdir(srcPath)
    fNum=0
    for fName in fNames:

        if os.path.exists(srcPath+'\\'+fName):
            # Loading image
            img = Image.open(srcPath+'\\'+fName)
            # Transform to tensor
            imgTensor = data_transforms(img).unsqueeze(0)
            if use_gpu:
                inp = Variable(imgTensor.cuda())
            else:
                inp = Variable(imgTensor)
            # Feed-forward through model+stack features for each video
            if fNum == 0:
                out = net(inp)                
                out = out.view(out.size()[0],-1).data.cpu()                
            else:
                out1 = net(inp)               
                out1 = out1.view(out1.size()[0],-1).data.cpu()                
                out = torch.cat((out,out1),0)
        else:
            print(fileName+ ' missing!') 
        fNum += 1
        
    # out dimension -> frame count x 512
    featSavePath = dir_name+"\\train\\"+cName # Directory for saving features
    if not os.path.exists(featSavePath):
        os.mkdir(featSavePath)
    torch.save(out,os.path.join(featSavePath,item+'.pt')) 
    print("Item {} feature extraction complete.".format(item))
    item_done.append(item)

end = time.time()
print("Train features extraction completed in {} mins {} secs".format((end-start)//60,int((end-start)%60)))    

Item v_ApplyEyeMakeup_g01_c01 feature extraction complete.
Item v_ApplyEyeMakeup_g01_c02 feature extraction complete.
Item v_ApplyEyeMakeup_g01_c03 feature extraction complete.
Item v_ApplyEyeMakeup_g01_c04 feature extraction complete.
Item v_ApplyEyeMakeup_g01_c05 feature extraction complete.
Item v_ApplyEyeMakeup_g01_c06 feature extraction complete.
Item v_ApplyEyeMakeup_g02_c01 feature extraction complete.
Item v_ApplyEyeMakeup_g02_c02 feature extraction complete.
Item v_ApplyEyeMakeup_g02_c03 feature extraction complete.
Item v_ApplyEyeMakeup_g02_c04 feature extraction complete.
Item v_ApplyEyeMakeup_g03_c01 feature extraction complete.
Item v_ApplyEyeMakeup_g03_c02 feature extraction complete.
Item v_ApplyEyeMakeup_g03_c03 feature extraction complete.
Item v_ApplyEyeMakeup_g03_c04 feature extraction complete.
Item v_ApplyEyeMakeup_g03_c05 feature extraction complete.
Item v_ApplyEyeMakeup_g03_c06 feature extraction complete.
Item v_ApplyEyeMakeup_g04_c01 feature extraction complet

Item v_ApplyLipstick_g06_c01 feature extraction complete.
Item v_ApplyLipstick_g06_c02 feature extraction complete.
Item v_ApplyLipstick_g06_c03 feature extraction complete.
Item v_ApplyLipstick_g06_c04 feature extraction complete.
Item v_ApplyLipstick_g06_c05 feature extraction complete.
Item v_ApplyLipstick_g07_c01 feature extraction complete.
Item v_ApplyLipstick_g07_c02 feature extraction complete.
Item v_ApplyLipstick_g07_c03 feature extraction complete.
Item v_ApplyLipstick_g07_c04 feature extraction complete.
Item v_ApplyLipstick_g08_c01 feature extraction complete.
Item v_ApplyLipstick_g08_c02 feature extraction complete.
Item v_ApplyLipstick_g08_c03 feature extraction complete.
Item v_ApplyLipstick_g08_c04 feature extraction complete.
Item v_ApplyLipstick_g09_c01 feature extraction complete.
Item v_ApplyLipstick_g09_c02 feature extraction complete.
Item v_ApplyLipstick_g09_c03 feature extraction complete.
Item v_ApplyLipstick_g09_c04 feature extraction complete.
Item v_ApplyLi

Item v_Archery_g14_c04 feature extraction complete.
Item v_Archery_g15_c01 feature extraction complete.
Item v_Archery_g15_c02 feature extraction complete.
Item v_Archery_g15_c03 feature extraction complete.
Item v_Archery_g15_c04 feature extraction complete.
Item v_Archery_g15_c05 feature extraction complete.
Item v_Archery_g15_c06 feature extraction complete.
Item v_Archery_g15_c07 feature extraction complete.
Item v_Archery_g16_c01 feature extraction complete.
Item v_Archery_g16_c02 feature extraction complete.
Item v_Archery_g16_c03 feature extraction complete.
Item v_Archery_g16_c04 feature extraction complete.
Item v_Archery_g16_c05 feature extraction complete.
Item v_Archery_g17_c01 feature extraction complete.
Item v_Archery_g17_c02 feature extraction complete.
Item v_Archery_g17_c03 feature extraction complete.
Item v_Archery_g17_c04 feature extraction complete.
Item v_Archery_g18_c01 feature extraction complete.
Item v_Archery_g18_c02 feature extraction complete.
Item v_Arche

Item v_BalanceBeam_g02_c03 feature extraction complete.
Item v_BalanceBeam_g02_c04 feature extraction complete.
Item v_BalanceBeam_g03_c01 feature extraction complete.
Item v_BalanceBeam_g03_c02 feature extraction complete.
Item v_BalanceBeam_g03_c03 feature extraction complete.
Item v_BalanceBeam_g03_c04 feature extraction complete.
Item v_BalanceBeam_g04_c01 feature extraction complete.
Item v_BalanceBeam_g04_c02 feature extraction complete.
Item v_BalanceBeam_g04_c03 feature extraction complete.
Item v_BalanceBeam_g04_c04 feature extraction complete.
Item v_BalanceBeam_g05_c01 feature extraction complete.
Item v_BalanceBeam_g05_c02 feature extraction complete.
Item v_BalanceBeam_g05_c03 feature extraction complete.
Item v_BalanceBeam_g05_c04 feature extraction complete.
Item v_BalanceBeam_g06_c01 feature extraction complete.
Item v_BalanceBeam_g06_c02 feature extraction complete.
Item v_BalanceBeam_g06_c03 feature extraction complete.
Item v_BalanceBeam_g06_c04 feature extraction co

Item v_BandMarching_g11_c02 feature extraction complete.
Item v_BandMarching_g11_c03 feature extraction complete.
Item v_BandMarching_g11_c04 feature extraction complete.
Item v_BandMarching_g11_c05 feature extraction complete.
Item v_BandMarching_g11_c06 feature extraction complete.
Item v_BandMarching_g11_c07 feature extraction complete.
Item v_BandMarching_g12_c01 feature extraction complete.
Item v_BandMarching_g12_c02 feature extraction complete.
Item v_BandMarching_g12_c03 feature extraction complete.
Item v_BandMarching_g12_c04 feature extraction complete.
Item v_BandMarching_g12_c05 feature extraction complete.
Item v_BandMarching_g12_c06 feature extraction complete.
Item v_BandMarching_g13_c01 feature extraction complete.
Item v_BandMarching_g13_c02 feature extraction complete.
Item v_BandMarching_g13_c03 feature extraction complete.
Item v_BandMarching_g13_c04 feature extraction complete.
Item v_BandMarching_g13_c05 feature extraction complete.
Item v_BandMarching_g14_c01 fea

Item v_BaseballPitch_g15_c02 feature extraction complete.
Item v_BaseballPitch_g15_c03 feature extraction complete.
Item v_BaseballPitch_g15_c04 feature extraction complete.
Item v_BaseballPitch_g15_c05 feature extraction complete.
Item v_BaseballPitch_g15_c06 feature extraction complete.
Item v_BaseballPitch_g15_c07 feature extraction complete.
Item v_BaseballPitch_g16_c01 feature extraction complete.
Item v_BaseballPitch_g16_c02 feature extraction complete.
Item v_BaseballPitch_g16_c03 feature extraction complete.
Item v_BaseballPitch_g16_c04 feature extraction complete.
Item v_BaseballPitch_g16_c05 feature extraction complete.
Item v_BaseballPitch_g16_c06 feature extraction complete.
Item v_BaseballPitch_g17_c01 feature extraction complete.
Item v_BaseballPitch_g17_c02 feature extraction complete.
Item v_BaseballPitch_g17_c03 feature extraction complete.
Item v_BaseballPitch_g17_c04 feature extraction complete.
Item v_BaseballPitch_g17_c05 feature extraction complete.
Item v_Basebal

Item v_BasketballDunk_g01_c03 feature extraction complete.
Item v_BasketballDunk_g01_c04 feature extraction complete.
Item v_BasketballDunk_g01_c05 feature extraction complete.
Item v_BasketballDunk_g01_c06 feature extraction complete.
Item v_BasketballDunk_g01_c07 feature extraction complete.
Item v_BasketballDunk_g02_c01 feature extraction complete.
Item v_BasketballDunk_g02_c02 feature extraction complete.
Item v_BasketballDunk_g02_c03 feature extraction complete.
Item v_BasketballDunk_g02_c04 feature extraction complete.
Item v_BasketballDunk_g03_c01 feature extraction complete.
Item v_BasketballDunk_g03_c02 feature extraction complete.
Item v_BasketballDunk_g03_c03 feature extraction complete.
Item v_BasketballDunk_g03_c04 feature extraction complete.
Item v_BasketballDunk_g03_c05 feature extraction complete.
Item v_BasketballDunk_g03_c06 feature extraction complete.
Item v_BasketballDunk_g04_c01 feature extraction complete.
Item v_BasketballDunk_g04_c02 feature extraction complet

Item v_BenchPress_g06_c06 feature extraction complete.
Item v_BenchPress_g06_c07 feature extraction complete.
Item v_BenchPress_g07_c01 feature extraction complete.
Item v_BenchPress_g07_c02 feature extraction complete.
Item v_BenchPress_g07_c03 feature extraction complete.
Item v_BenchPress_g07_c04 feature extraction complete.
Item v_BenchPress_g07_c05 feature extraction complete.
Item v_BenchPress_g07_c06 feature extraction complete.
Item v_BenchPress_g07_c07 feature extraction complete.
Item v_BenchPress_g08_c01 feature extraction complete.
Item v_BenchPress_g08_c02 feature extraction complete.
Item v_BenchPress_g08_c03 feature extraction complete.
Item v_BenchPress_g08_c04 feature extraction complete.
Item v_BenchPress_g08_c05 feature extraction complete.
Item v_BenchPress_g08_c06 feature extraction complete.
Item v_BenchPress_g08_c07 feature extraction complete.
Item v_BenchPress_g09_c01 feature extraction complete.
Item v_BenchPress_g09_c02 feature extraction complete.
Item v_Ben

In [13]:
framePath = "C:\\Users\\smitr\\Desktop\\test_10class\\"
start = time.time()

for item in testList:
    cName = item.split('_')[1]
    srcPath = framePath+cName+'\\'+item    
    fNames = os.listdir(srcPath)
    fNum = 0
    for fName in fNames:
        img_path = srcPath+'\\'+fName
        if os.path.exists(img_path):
            img = Image.open(img_path)
            imgTensor = data_transforms(img).unsqueeze(0)
            inp = Variable(imgTensor.cuda())
            if fNum == 0:
                out = net(inp)                
                out = out.view(out.size()[0],-1).data.cpu()
            else:
                out1 = net(inp)               
                out1 = out1.view(out1.size()[0],-1).data.cpu()                
                out = torch.cat((out,out1),0)
        else:
            print(fileName+ ' missing!')
        fNum += 1
      
    featSavePath = dir_name+"\\test\\"+cName
    if not os.path.exists(featSavePath):
        os.mkdir(featSavePath)
    torch.save(out,os.path.join(featSavePath,item+'.pt'))   
    print("Item {} feature extraction complete.".format(item))

end = time.time()
print("Test features extraction completed in {} mins {} secs".format((end-start)//60,int((end-start)%60)))

Item v_ApplyEyeMakeup_g21_c02 feature extraction complete.
Item v_ApplyEyeMakeup_g21_c03 feature extraction complete.
Item v_ApplyEyeMakeup_g21_c04 feature extraction complete.
Item v_ApplyEyeMakeup_g21_c05 feature extraction complete.
Item v_ApplyEyeMakeup_g22_c01 feature extraction complete.
Item v_ApplyEyeMakeup_g22_c02 feature extraction complete.
Item v_ApplyEyeMakeup_g22_c03 feature extraction complete.
Item v_ApplyEyeMakeup_g22_c04 feature extraction complete.
Item v_ApplyEyeMakeup_g22_c05 feature extraction complete.
Item v_ApplyEyeMakeup_g23_c01 feature extraction complete.
Item v_ApplyEyeMakeup_g23_c02 feature extraction complete.
Item v_ApplyEyeMakeup_g23_c03 feature extraction complete.
Item v_ApplyEyeMakeup_g23_c04 feature extraction complete.
Item v_ApplyEyeMakeup_g23_c05 feature extraction complete.
Item v_ApplyEyeMakeup_g23_c06 feature extraction complete.
Item v_ApplyEyeMakeup_g24_c01 feature extraction complete.
Item v_ApplyEyeMakeup_g24_c02 feature extraction complet

Item v_BandMarching_g23_c04 feature extraction complete.
Item v_BandMarching_g23_c05 feature extraction complete.
Item v_BandMarching_g23_c06 feature extraction complete.
Item v_BandMarching_g24_c01 feature extraction complete.
Item v_BandMarching_g24_c02 feature extraction complete.
Item v_BandMarching_g24_c03 feature extraction complete.
Item v_BandMarching_g24_c04 feature extraction complete.
Item v_BandMarching_g24_c05 feature extraction complete.
Item v_BandMarching_g24_c06 feature extraction complete.
Item v_BandMarching_g25_c01 feature extraction complete.
Item v_BandMarching_g25_c02 feature extraction complete.
Item v_BandMarching_g25_c03 feature extraction complete.
Item v_BandMarching_g25_c04 feature extraction complete.
Item v_BandMarching_g25_c05 feature extraction complete.
Item v_BandMarching_g25_c06 feature extraction complete.
Item v_BandMarching_g25_c07 feature extraction complete.
Item v_BaseballPitch_g21_c03 feature extraction complete.
Item v_BaseballPitch_g21_c04 f